In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

For this notebook, use the hints to fill in the missing code.

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

# We'll just read in data with these prefixes so we have easier data to deal with
# These are "target" prefixes
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# Only use pairs where the english data (pair[1]) has the prefix above
# Also, only consider data where pair[0] and pair[1] have length less than MAX_LENGTH
# Split on space first here also as MAX_LENGTH means the number of tokens (words), not number of chars
def filterPair(p):
    return p[1].startswith(eng_prefixes) and len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!unzip 'data.zip'

Archive:  /content/drive/MyDrive/Deep Learning for NLP Spring 2023/HW9/data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [12]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis l une de vos eleves .', 'i m one of your students .']


In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # Initialize the embedding and lstm; use batch_first=True
        self.embedding = nn.Embedding(input_size, self.hidden_size).to(device)
        self.lstm = nn.LSTM(input_size = self.hidden_size, hidden_size = self.hidden_size, batch_first = True).to(device)

    def forward(self, input, hidden_cell):
        # Get the embeddings and reshape to be (1, 1, -1)
        embedded = self.embedding(input)
        output = embedded.view(1, 1, -1)
        # Push through the lstm
        output, hidden_cell = self.lstm(output, hidden_cell)
        return output, hidden_cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # Initialize the embedding
        self.embedding = nn.Embedding(output_size, self.hidden_size).to(device)
        # Initialize an LSTM with yt and kt dimensions hidden_size
        # Use batch_first=True
        self.lstm = nn.LSTM(input_size = self.hidden_size, hidden_size = self.hidden_size, batch_first = True).to(device)
        # Initialize a Linear layer going to the appropriate vocabulary size
        self.out = nn.Linear(self.hidden_size, output_size).to(device)
        self.softmax = nn.LogSoftmax(dim = 2).to(device)

    def forward(self, input, hidden_cell):
        output = self.embedding(input).view(1, 1, -1)
        # Push through relu
        output = nn.ReLU()(output)
        # Push output and hidden_cell tuple through the lstm
        output, hidden_cell = self.lstm(output, hidden_cell)
        output = self.out(output)
        output = self.softmax(output)
        return output, hidden_cell

    def initHidden(self):
        # Used to initialize the hidden state (or cell state) to a tensor of dimension (1, 1, hidden_size)
        # Just return a tensor here
        return torch.zeros(1, 1, self.hidden_size, device = device)

In [15]:
# Split a sentence by ' ' and return a list of the tokens (int ids) for each word
# Use word2index
def indexesFromSentence(lang, sentence):
    return [lang.word2index[s] for s in sentence.split(' ')]

# Call the above on a sentence
# After calling, add the EOS_token (int id) to the gotten list
# Return a tensor, but reshape it so it's dimensions (-1, 1)
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes).view(-1, 1).to(device)

# For a source, target pair, call the above. Return a tuple of 2 tensors, one input_tensor and another an output_tensor
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [16]:
teacher_forcing_ratio = 0.5

def train(
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    max_length=MAX_LENGTH
):
    
    # Initialize the hidden and cell states
    encoder_hidden = encoder.initHidden()
    encoder_cell = encoder.initHidden()

    # Reset the optimizer gradients to 0
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0
    # Reverse the input here, see below
    input_tensor = torch.flip(input_tensor, dims = [0]).to(device)
    
    # If we want to predict [x, y, z] from [a, b, c], we should feed in [c, b, a] on the encoder side so that a is as close to x as possible
    # The above trick was noted in making a big difference
    for it in range(input_length):
        encoder_output, (encoder_hidden, encoder_cell) = encoder(input_tensor[it] ,(encoder_hidden, encoder_cell))

    # Initialize the decoder input to the SOS_token
    decoder_input = torch.tensor([[SOS_token]], device = device)

    # Initialize the hidden states of the decoder with the hidden states of the encoder
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    # For this pair, use teacher forcing with 50% probability, else don't
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    
    target_length_used = 0

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        target_length_used = target_length
        
        for jt in range(target_length):
            # Push decoder_input, decoder_hidden, and decoder_cell through the decoder
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            # Update the loss
            loss += criterion(decoder_output.squeeze(0), target_tensor[jt].reshape(-1))
            # Set the next decoder_input to the current y_t
            decoder_input = target_tensor[jt].unsqueeze(0) # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        # Note that in this case we feed in at most target_length words
        # If, however, we predict EOS_token, we break out
        # You need to also carefully get the target lenght used since it might not be target_length
        for jt in range(target_length):
            # As before
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            
            # Get the top index, \hat{y}_t; this will be the next decoder_input
            topv, topi = torch.topk(decoder_output, k = 1, dim = -1)
            decoder_input = topi.squeeze().detach()  # Detach from history as input

            loss += criterion(decoder_output.squeeze(0), target_tensor[jt].reshape(-1))
            
            # Update the target_length_used
            target_length_used += 1
            
            # If the EOS_token was generated, exit
            if decoder_input.item() == EOS_token:
                break

    # Collect gradients
    loss.backward()

    # Do a step; do this both for the encoder and the decoder
    encoder_optimizer.step()
    decoder_optimizer.step()

    # Return the loss for this pair. Note that you'll need to use target_length_used not target_length; why?
    return loss.item()/target_length_used

In [17]:
# Helper functions used to time the optimizations
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

trainIters below does SGD (batch 1) on the data.

In [18]:
import matplotlib.pyplot as plt, smoothing_function=chencherry
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def trainIters(
    encoder,
    decoder,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01
):
    print('trainIters')

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # Get n_iters training pairs
    # In this example, we are effectively doing SGD with batch size 1
    training_pairs = [tensorsFromPair(p) for p in random.choices(pairs, k = n_iters)]
    
    # The loss; either NLLLoss if you use log sigmoids or CrossEntropyLoss if you use logits
    criterion = nn.NLLLoss()

    for it in range(1, n_iters + 1):
        
        training_pair = training_pairs[it - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # Train on the input, target pair
        loss = train(input_tensor,
                    target_tensor,
                    encoder,
                    decoder,
                    encoder_optimizer,
                    decoder_optimizer,
                    criterion,
                    max_length=MAX_LENGTH)
        
        # Update the total loss and the plot loss
        # We can plot and print at different granularities
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(
                '%s (%d %d%%) %.4f' % (
                    timeSince(start, it / n_iters),
                    it, it / n_iters * 100, print_loss_avg)
            )

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            showPlot(plot_losses)

In [19]:
hidden_size = 256
cell_size = 256
# Initialize the encoder and decoder and run them through the trainIters function
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder, decoder, 75000, print_every=5000)

trainIters


<ipython-input-18-f2ab747b8554>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


0m 56s (- 13m 11s) (5000 6%) 3.1168
1m 49s (- 11m 50s) (10000 13%) 2.4958
2m 43s (- 10m 52s) (15000 20%) 2.2025
3m 35s (- 9m 53s) (20000 26%) 1.9572
4m 28s (- 8m 56s) (25000 33%) 1.7728
5m 22s (- 8m 3s) (30000 40%) 1.6143
6m 14s (- 7m 8s) (35000 46%) 1.4699
7m 7s (- 6m 14s) (40000 53%) 1.3219
8m 0s (- 5m 20s) (45000 60%) 1.2125
8m 55s (- 4m 27s) (50000 66%) 1.0917
9m 47s (- 3m 33s) (55000 73%) 1.0046
10m 41s (- 2m 40s) (60000 80%) 0.9193
11m 34s (- 1m 46s) (65000 86%) 0.8200
12m 26s (- 0m 53s) (70000 93%) 0.7658
13m 18s (- 0m 0s) (75000 100%) 0.7224


Fill in the evaluation function using the hints below.

In [22]:
# For a certain input, get the predicted output sentence
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # Transform the input sentence into a tensor
        input_tensor = tensorFromSentence(input_lang, sentence).to(device)
        
        input_length = input_tensor.size()[0]
        # Initilize the hidden and cell states of the LSTM
        encoder_hidden = encoder.initHidden()
        encoder_cell = encoder.initHidden()
        
        # Run the data through the LSTM word by word manually
        # At each step, feed in the input, the hidden state, and the cell state and calture the new hidden / cell states
        for it in range(input_length):
            encoder_output, (encoder_hidden, encoder_cell) = encoder(input_tensor[it], (encoder_hidden, encoder_cell))
        
        # Initialize the decoder input with the SOS token
        # This is y_0
        decoder_input = torch.tensor([[SOS_token]], device = device)  # SOS

        # Initialize the decoder hidden and cell states with the final encoder hidden and cell states
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        decoded_words = []

        for jt in range(max_length):
            # As with the encoder run the \hat{y}_{t-1}, hidden, and state cells through the decoder
            # Capture the new hidden and cell states and the logits or log_softmax for the vocabulary
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            # Get the top y for the decoder, this will be the new \hat{y}_t which we can use at the next step
            _, topi = torch.topk(decoder_output, k = 1, dim = -1)
            
            # Put logic so that if we get topi == EOS_token, we add this and break
            # Otherwise, we map the index topi to the word in output_lang via index2word
            topi = topi.squeeze().detach().item()
            if topi == EOS_token:
                decoded_words.append(output_lang.index2word[topi])
                break
            else:
                decoded_words.append(output_lang.index2word[topi])

            decoder_input = torch.tensor([topi], device = device)

        return decoded_words

In [29]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluateRandomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        # Randomly choose a pair of sentences
        pair = pairs[random.randint(1, len(pairs)-1)]
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
        # Leave out the <EOS> symbol
        # Run the source French sentence through the encoder-decoder and get the output_words
        output_words = evaluate(encoder, decoder, pair[0])
        
        # If <EOS> is at the end of output_words, remove it
        output_words = output_words[:-1]
                
        output_sentence = ' '.join(output_words)
        # Get the BLEU score based on 1 and 2 grams (words, bigrams); use 50% weight on each
        # Use pair[1] as the reference
        chencherry = SmoothingFunction().method1
        score = sentence_bleu([pair[1].split(' ')], output_words, weights = (0.5, 0.5), smoothing_function=chencherry)*100
        # Append the BLEU score you got to the list of BLEU scores you keep
        bleu_scores.append(score)
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    # Return the mean of the BLEU scores
    print('The mean BLEU score is: ', np.mean(bleu_scores))

In [30]:
evaluateRandomly(encoder, decoder)

The mean BLEU score is:  14.799655712897058
